In [ ]:
!pip install yfinance

In [ ]:
pip install config

In [ ]:
pip install websocket-client

In [ ]:
!pip install alpaca-trade-api

In [ ]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
import os
from dotenv import load_dotenv
import requests
import json
from getpass import getpass
import alpaca_trade_api as alpacaapi
import websocket
import _thread
import time
import yfinance as yf
from sklearn.linear_model import LogisticRegression

In [ ]:
bitcoin = yf.Ticker("BTC-USD")
#current_btc = bitcoin.history(period='1d', interval="1m")[['Close']]
Bitcoin_df = bitcoin.history(start="2017-12-31", end="2021-12-07")

In [ ]:
ethereum = yf.Ticker("ETH-USD")
#current_btc = bitcoin.history(period='1d', interval="1m")[['Close']]
Ethereum_df = ethereum.history(start="2017-12-31", end="2021-12-07")

In [ ]:
litecoin = yf.Ticker("LTC-USD")
#current_btc = bitcoin.history(period='1d', interval="1m")[['Close']]
LiteCoin_df = litecoin.history(start="2017-12-31", end="2021-12-07")

In [ ]:
bitcoincash = yf.Ticker("BCH-USD")
BitcoinCash_df = bitcoincash.history(start="2017-12-31", end="2021-12-07")

In [ ]:
BitcoinCash_df.head()

In [ ]:
# separate into individual dataframes

btc_signals_df=Bitcoin_df.loc[:, ["Close"]]
eth_signals_df=Ethereum_df.loc[:, ["Close"]]
ltc_signals_df=LiteCoin_df.loc[:, ["Close"]]
bch_signals_df=BitcoinCash_df.loc[:,["Close"]]

display(btc_signals_df.tail())
display(eth_signals_df.head())
display(ltc_signals_df.head())
display(bch_signals_df.head())

In [ ]:
# use pct_change function to get actual returns and dropna

btc_signals_df["Actual Returns"]=btc_signals_df["Close"].pct_change()
eth_signals_df["Actual Returns"]=eth_signals_df["Close"].pct_change()
ltc_signals_df["Actual Returns"]=ltc_signals_df["Close"].pct_change()
bch_signals_df["Actual Returns"]=bch_signals_df["Close"].pct_change()

btc_signals_df=btc_signals_df.dropna()
eth_signals_df=eth_signals_df.dropna()
ltc_signals_df=ltc_signals_df.dropna()
bch_signals_df=bch_signals_df.dropna()

display(btc_signals_df.head())
display(eth_signals_df.head())
display(ltc_signals_df.head())
display(bch_signals_df.head())

In [ ]:
# Set the short window and long window
short_window = 30
long_window = 120

# Generate the fast and slow simple moving averages 

btc_signals_df['SMA_Fast']=btc_signals_df['Close'].rolling(window=short_window).mean()
btc_signals_df['SMA_Slow']=btc_signals_df['Close'].rolling(window=long_window).mean()

In [ ]:
short_window = 30
long_window = 90

eth_signals_df['SMA_Fast']=eth_signals_df['Close'].rolling(window=short_window).mean()
eth_signals_df['SMA_Slow']=eth_signals_df['Close'].rolling(window=long_window).mean()

In [ ]:
short_window = 30
long_window = 90

ltc_signals_df['SMA_Fast']=ltc_signals_df['Close'].rolling(window=short_window).mean()
ltc_signals_df['SMA_Slow']=ltc_signals_df['Close'].rolling(window=long_window).mean()

In [ ]:
short_window = 30
long_window = 90

bch_signals_df['SMA_Fast']=bch_signals_df['Close'].rolling(window=short_window).mean()
bch_signals_df['SMA_Slow']=bch_signals_df['Close'].rolling(window=long_window).mean()

In [ ]:
#check your work
display(btc_signals_df.head())
display(eth_signals_df.head())
display(ltc_signals_df.head())
display(bch_signals_df.head())

In [ ]:
# add the  Signal column
btc_signals_df['Signal']=0.0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock 

btc_signals_df.loc[(btc_signals_df['Actual Returns'] >= 0), 'Signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock

btc_signals_df.loc[(btc_signals_df['Actual Returns'] < 0), 'Signal'] = -1

In [ ]:
eth_signals_df['Signal']=0.0
eth_signals_df.loc[(eth_signals_df['Actual Returns'] >= 0), 'Signal'] = 1
eth_signals_df.loc[(eth_signals_df['Actual Returns'] < 0), 'Signal'] = -1

In [ ]:
ltc_signals_df['Signal']=0.0
ltc_signals_df.loc[(ltc_signals_df['Actual Returns'] >= 0), 'Signal'] = 1
ltc_signals_df.loc[(ltc_signals_df['Actual Returns'] < 0), 'Signal'] = -1

In [ ]:
bch_signals_df['Signal']=0.0
bch_signals_df.loc[(bch_signals_df['Actual Returns'] >= 0), 'Signal'] = 1
bch_signals_df.loc[(bch_signals_df['Actual Returns'] < 0), 'Signal'] = -1

In [ ]:
# check your work
display(btc_signals_df.head())
display(eth_signals_df.head())
display(ltc_signals_df.head())
display(bch_signals_df.head())

display(btc_signals_df.tail())
display(eth_signals_df.tail())
display(ltc_signals_df.tail())
display(bch_signals_df.tail())

In [ ]:
# Add Strategy Returns column
btc_signals_df['Strategy Returns'] = btc_signals_df['Actual Returns'] * btc_signals_df['Signal'].shift()

In [ ]:
eth_signals_df['Strategy Returns'] = eth_signals_df['Actual Returns'] * eth_signals_df['Signal'].shift()

In [ ]:
ltc_signals_df['Strategy Returns'] = ltc_signals_df['Actual Returns'] * ltc_signals_df['Signal'].shift()

In [ ]:
bch_signals_df['Strategy Returns'] = bch_signals_df['Actual Returns'] * bch_signals_df['Signal'].shift()

In [ ]:
# check your work
display(btc_signals_df.head())
display(eth_signals_df.head())
display(ltc_signals_df.head())
display(bch_signals_df.head())

display(btc_signals_df.tail())
display(eth_signals_df.tail())
display(ltc_signals_df.tail())
display(bch_signals_df.tail())

In [ ]:
# create X and y variables
btc_X = btc_signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()
btc_y = btc_signals_df['Signal']

In [ ]:
eth_X = eth_signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()
eth_y = eth_signals_df['Signal']

In [ ]:
ltc_X = ltc_signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()
ltc_y = ltc_signals_df['Signal']

In [ ]:
bch_X = bch_signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()
bch_y = bch_signals_df['Signal']

In [ ]:
# set training start and end times
btc_training_begin = btc_X.index.min()
btc_training_end = btc_X.index.min() + DateOffset(months=8)

In [ ]:
eth_training_begin = eth_X.index.min()
eth_training_end = eth_X.index.min() + DateOffset(months=8)

In [ ]:
ltc_training_begin = ltc_X.index.min()
ltc_training_end = ltc_X.index.min() + DateOffset(months=8)

In [ ]:
bch_training_begin = bch_X.index.min()
bch_training_end = bch_X.index.min() + DateOffset(months=8)

In [ ]:
# create train and test variables
btc_X_train = btc_X.loc[btc_training_begin:btc_training_end]
btc_y_train = btc_y.loc[btc_training_begin:btc_training_end]

btc_X_test = btc_X.loc[btc_training_end+DateOffset(hours=1):]
btc_y_test = btc_y.loc[btc_training_end+DateOffset(hours=1):]

In [ ]:
eth_X_train = eth_X.loc[eth_training_begin:eth_training_end]
eth_y_train = eth_y.loc[eth_training_begin:eth_training_end]

eth_X_test = eth_X.loc[eth_training_end+DateOffset(hours=1):]
eth_y_test = eth_y.loc[eth_training_end+DateOffset(hours=1):]

In [ ]:
ltc_X_train = ltc_X.loc[ltc_training_begin:ltc_training_end]
ltc_y_train = ltc_y.loc[ltc_training_begin:ltc_training_end]

ltc_X_test = ltc_X.loc[ltc_training_end+DateOffset(hours=1):]
ltc_y_test = ltc_y.loc[ltc_training_end+DateOffset(hours=1):]

In [ ]:
bch_X_train = bch_X.loc[bch_training_begin:bch_training_end]
bch_y_train = bch_y.loc[bch_training_begin:bch_training_end]

bch_X_test = bch_X.loc[bch_training_end+DateOffset(hours=1):]
bch_y_test = bch_y.loc[bch_training_end+DateOffset(hours=1):]

In [ ]:
# scale the features
btc_scaler = StandardScaler()
btc_X_scaler = btc_scaler.fit(btc_X_train)
btc_X_train_scaled = btc_X_scaler.transform(btc_X_train)
btc_X_test_scaled = btc_X_scaler.transform(btc_X_test)

In [ ]:
eth_scaler = StandardScaler()
eth_X_scaler = eth_scaler.fit(eth_X_train)
eth_X_train_scaled = eth_X_scaler.transform(eth_X_train)
eth_X_test_scaled = eth_X_scaler.transform(eth_X_test)

In [ ]:
ltc_scaler = StandardScaler()
ltc_X_scaler = ltc_scaler.fit(ltc_X_train)
ltc_X_train_scaled = ltc_X_scaler.transform(ltc_X_train)
ltc_X_test_scaled = ltc_X_scaler.transform(ltc_X_test)

In [ ]:
bch_scaler = StandardScaler()
bch_X_scaler = bch_scaler.fit(bch_X_train)
bch_X_train_scaled = bch_X_scaler.transform(bch_X_train)
bch_X_test_scaled = bch_X_scaler.transform(bch_X_test)

In [ ]:
# use SKLearn SVC classifier to fit and predict
btc_model = LogisticRegression()
btc_model = btc_model.fit(btc_X_train_scaled, btc_y_train)
btc_pred = btc_model.predict(btc_X_test_scaled)
btc_pred

In [ ]:
eth_model = LogisticRegression()
eth_model = eth_model.fit(eth_X_train_scaled, eth_y_train)
eth_pred = eth_model.predict(eth_X_test_scaled)
eth_pred

In [ ]:
ltc_model = LogisticRegression()
ltc_model = ltc_model.fit(ltc_X_train_scaled, ltc_y_train)
ltc_pred = ltc_model.predict(ltc_X_test_scaled)
ltc_pred

In [ ]:
bch_model = LogisticRegression()
bch_model = bch_model.fit(bch_X_train_scaled, bch_y_train)
bch_pred = bch_model.predict(bch_X_test_scaled)
bch_pred

In [ ]:
# review the classification report
btc_testing_report = classification_report(btc_y_test, btc_pred)
print(btc_testing_report)

In [ ]:
eth_testing_report = classification_report(eth_y_test, eth_pred)
print(eth_testing_report)

In [ ]:
ltc_testing_report = classification_report(ltc_y_test, ltc_pred)
print(ltc_testing_report)

In [ ]:
bch_testing_report = classification_report(bch_y_test, bch_pred)
print(bch_testing_report)

In [ ]:
# Create a new data frame to compare Actual Returns and Strategy Returns
btc_predictions_df = pd.DataFrame(index=btc_X_test.index)
btc_predictions_df['Predicted'] = btc_pred
btc_predictions_df['Actual Returns'] = btc_signals_df['Actual Returns']
btc_predictions_df['Strategy Returns'] = (
    btc_predictions_df['Actual Returns'] * btc_predictions_df['Predicted'])
display(btc_predictions_df.tail(100))

In [ ]:
btc_predictions_df["Predicted"].plot()

In [ ]:
eth_predictions_df = pd.DataFrame(index=eth_X_test.index)
eth_predictions_df['Predicted'] = eth_pred
eth_predictions_df['Actual Returns'] = eth_signals_df['Actual Returns']
eth_predictions_df['Strategy Returns'] = (
    eth_predictions_df['Actual Returns'] * eth_predictions_df['Predicted'])
display(eth_predictions_df.head())
display(eth_predictions_df.tail())

In [ ]:
ltc_predictions_df = pd.DataFrame(index=ltc_X_test.index)
ltc_predictions_df['Predicted'] = ltc_pred
ltc_predictions_df['Actual Returns'] = ltc_signals_df['Actual Returns']
ltc_predictions_df['Strategy Returns'] = (
    ltc_predictions_df['Actual Returns'] * ltc_predictions_df['Predicted'])
display(ltc_predictions_df.head())
display(ltc_predictions_df.tail())

In [ ]:
bch_predictions_df = pd.DataFrame(index=bch_X_test.index)
bch_predictions_df['Predicted'] = bch_pred
bch_predictions_df['Actual Returns'] = bch_signals_df['Actual Returns']
bch_predictions_df['Strategy Returns'] = (
    bch_predictions_df['Actual Returns'] * bch_predictions_df['Predicted'])
display(bch_predictions_df.head())
display(bch_predictions_df.tail())

In [ ]:
# Plot the actual vs strategy returns
btc_plot=(1+ btc_predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().plot(
title='Actual Returns vs Strategy Returns - Bitcoin')

btc_plot

In [ ]:
eth_plot=(1+ eth_predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().plot(
title='Actual Returns vs Strategy Returns - Ethereum')

eth_plot

In [ ]:
ltc_plot=(1+ ltc_predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().plot(
title='Actual Returns vs Strategy Returns - Litecoin')

ltc_plot

In [ ]:
bch_plot=(1+ bch_predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().plot(
title='Actual Returns vs Strategy Returns - BitcoinCash')
bch_plot

In [ ]:
# add entry/exit points to dataframe
btc_predictions_df["Entry/Exit"] = btc_predictions_df["Predicted"].diff()


In [ ]:
eth_predictions_df["Entry/Exit"] = eth_predictions_df["Predicted"].diff()


In [ ]:
ltc_predictions_df["Entry/Exit"] = ltc_predictions_df["Predicted"].diff()


In [ ]:
bch_predictions_df["Entry/Exit"] = bch_predictions_df["Predicted"].diff()


In [ ]:
alp_api = 
secret_key1 = 

In [ ]:
api = alpacaapi.REST(alp_api, secret_key1, "https://paper-api.alpaca.markets")

In [ ]:
# Fetch Account
account = api.get_account()

# Print Account Details
print(account.id, account.equity, account.status)

In [ ]:
btc_pred_only = btc_predictions_df.loc[:, ["Entry/Exit"]]
eth_pred_only = eth_predictions_df.loc[:, ["Entry/Exit"]]
ltc_pred_only = ltc_predictions_df.loc[:, ["Entry/Exit"]]
bch_pred_only = bch_predictions_df.loc[:, ["Entry/Exit"]]

In [ ]:
btc_predictions_df.tail()

In [ ]:
btc_pred_only.head(50)

In [ ]:
#Instantiate buy order for all
api.submit_order(symbol="BTCUSD", qty=1,
    side='buy',
    type='market',
    time_in_force='day')
api.submit_order(symbol="ETHUSD", qty=1,
    side='buy',
    type='market',
    time_in_force='day')
api.submit_order(symbol="LTCUSD", qty=1,
    side='buy',
    type='market',
    time_in_force='day')
api.submit_order(symbol="BCHUSD", qty=1,
    side='buy',
    type='market',
    time_in_force='day')

In [ ]:
if btc_pred_only["Entry/Exit"].iloc[-1] == 2.0:
    print("buy")
    api.submit_order(symbol='BTCUSD',
    qty=1,
    side='buy',
    type='market',
    time_in_force='day')
elif btc_pred_only["Entry/Exit"].iloc[-1]== -2.0:
    print("sell")
    api.submit_order(symbol='BTCUSD',
                      qty=1,
                      side='sell',
                      type='market',
                      time_in_force='day')
else: print("hold")    

In [ ]:
if eth_pred_only["Entry/Exit"].iloc[-1] == 2.0:
    print("buy")
    api.submit_order(symbol='ETHUSD',
    qty=1,
    side='buy',
    type='market',
    time_in_force='day')
elif eth_pred_only["Entry/Exit"].iloc[-1] == -2.0:
    print("sell")
    api.submit_order(symbol='ETHUSD',
                      qty=1,
                      side='sell',
                      type='market',
                      time_in_force='day')
else: print("hold")    

In [ ]:
if ltc_pred_only["Entry/Exit"].iloc[-1] == 2.0:
    print("buy")
    api.submit_order(symbol='LTCUSD',
    qty=1,
    side='buy',
    type='market',
    time_in_force='day')
elif ltc_pred_only["Entry/Exit"].iloc[-1] == -2.0:
    print("sell")
    api.submit_order(symbol='LTCUSD',
                      qty=1,
                      side='sell',
                      type='market',
                      time_in_force='day')
else: print("hold")    

In [ ]:
if bch_pred_only["Entry/Exit"].iloc[-1] == 2.0:
    print("buy")
    api.submit_order(symbol='BCHUSD',
    qty=1,
    side='buy',
    type='market',
    time_in_force='day')
elif bch_pred_only["Entry/Exit"].iloc[-1] == -2.0:
    print("sell")
    api.submit_order(symbol='BCHUSD',
                      qty=1,
                      side='sell',
                      type='market',
                      time_in_force='day')
else: print("hold")    